#  텐서플로를 사용한 사용자 정의 모델과 훈련
### 텐서플로 훑어보기
* 넘파이와 비슷하나 GPU지원
* 분산 컴퓨팅 지원
* 속도를 높이고 메모리 사용량을 줄이기 위해 계산을 최적화한다. 이를 위해 계산 그래프를 추출한 다음 최적화한다.
* 계산그래프는 플랫폼에 중립적인 포맷으로 내보낼 수 있다.(리눅스 훈련 후 안드로이드에서 실행)
* 자동 미분과 고성능 옵티마이저 제공한다.
* 여러 디바이스와 서버로 이루어진 분산환경에서도 효율적으로 실행한다.
* 광범위한 라이브러리 생태계를 가진다 
  * 시각화를 위한 텐서보드
  * 텐서플로 제품화를 위한 라이브러리 모음인 TFX
  * 사전훈련된 신경망을 사용하는 텐서플로 허브
  * 텐서플로 모델 저장소에서 많은 신경망 구조를 다운로드 할 수 있다.
  * 텐서플로 리소스 페이지등 프로젝트 확인 가능.
___
## 넘파이처럼 텐서플로 사용하기
* 텐서플로는 텐서를 순환시킨다. 텐서는 넘파이와 비슷하다.
### 텐서와 연산
* tf.constant()함수로 텐서를 만들 수 있다.
  * 크기(shape)와 타입(dtype)이 있다.

In [16]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
t=tf.constant([[1.,2,3],[4,5,6]])
t.shape,t,t.dtype

(TensorShape([2, 3]), <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1., 2., 3.],
        [4., 5., 6.]], dtype=float32)>, tf.float32)

* 인덱스 참조도 넘파이와 비슷하다.

In [7]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [11]:
t[...,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

* 모든 종류의 텐서 연산이 가능하다.
  * @연산은 행렬 곱셈을 위해 파이썬에 추가됨. tf.matmul()함수 호출과 동일하다.
  * 넘파이의 기본 수학 연산 사용 가능
    * tf.add(), tf.multyply)(), tf.square(), tf.exp(), tf.sqrt(), tf.reshape(), tf.squeeze(), tf.tile() 제공
  * 일부 함수는 넘파이와 다르다
    * tf.reduce_mean(), tf.reduce_sum(), tf.reduce_max(), tf.math.log()
      * tf.transpose(t)

In [12]:
t+10,tf.square(t),t@tf.transpose(t)

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[11., 12., 13.],
        [14., 15., 16.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[ 1.,  4.,  9.],
        [16., 25., 36.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[14., 32.],
        [32., 77.]], dtype=float32)>)

* 케라스 저수준 API
  * keras.backend에 자체적 저수준 API를 가지고있다.

In [14]:
from tensorflow import keras
K=keras.backend
K.square(K.transpose(t))+10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

##### 텐서와 넘파이
* 텐서는 넘파이와 함께 사용하기 편리하다
* 텐서와 넘파이가 서로 호환이된다.

In [18]:
a=np.array([2.,4.,5.])
tf.constant(a),t.numpy(),tf.square(a),np.square(t)

(<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>,
 array([[1., 2., 3.],
        [4., 5., 6.]], dtype=float32),
 <tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>,
 array([[ 1.,  4.,  9.],
        [16., 25., 36.]], dtype=float32))

### 타입 변환
* 텐서플로는 어떤 타입 변환도 자동으로 수행하지 않는다.
* 호환되지 않는 타입의 텐서로 연산을 실행하면 예외가 발생한다.
* 64비트 실수와 32비트 실수 연산도 안된다.
* cast로 타입변환 가능

In [24]:
tf.constant(2.)+tf.cast(tf.constant(40.,dtype=tf.float64),tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

##### 변수
* tf.tensor는 변경이 불가능한 객체이다. 즉 텐서의 내용을 바꿀수 없다
* 이는 역전파 신경망의 가중치 구할 수 없다.
* tf.Variable는 assign메서드로 변숫값 변환 가능
  * assign_add,assign_sub 로 증감 가능
  * scatter_update(),scatter_nd_update 메서드로 개별 원소 수정도 가능
  

In [32]:
v=tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [31]:
v.assign(2*v),v[0,1].assign(42),v[:,2].assign([0.,1.]),v

(<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
 array([[ 2., 42.,  0.],
        [ 8., 10.,  1.]], dtype=float32)>,
 <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
 array([[ 2., 42.,  0.],
        [ 8., 10.,  1.]], dtype=float32)>,
 <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
 array([[ 2., 42.,  0.],
        [ 8., 10.,  1.]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
 array([[ 2., 42.,  0.],
        [ 8., 10.,  1.]], dtype=float32)>)

In [33]:
v.scatter_nd_update(indices=[[0,0],[1,2]],updates=[100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   2.,   3.],
       [  4.,   5., 200.]], dtype=float32)>

##### 다른 데이터 구조
* 희소 텐서 (tf.SparseTensor) -> 대부분 0으로 체워진 탠서를 효율적으로 나타낸다.
* 텐서 배열(tf.TensorArray) -> 텐서 리스트. 고정적 길이를 가지지만 동적으로 바꿀 수 있다.
* 래그드 텐서(tf.RaggedTensor) ->리스트의 리스트를 나타낸다. 동일한 데이터 타입을 가지지만 길이는 다를수 있다.
* 문자열 텐서(string tensor) -> tf.string 타입의 텐서이다.
* 집합(set) -> 일반적인 텐서로 나타난ㄷ.
* 큐(queue) -> 단계별로 텐서를 저장한다
---
## 사용자 정의 모델과 훈련 알고리즘
### 사용자 정의 손실 함수
* 잡음이 조금있을때 mse는 너무 과한 벌칙이 된다. mae는 이상치에 관대해서 수렴까지 시간이 걸린다.
* 이경우 후버 손실을 사용하면 좋은데 아직 공식 케라스api에서 지원하지 않는다.

In [34]:
def huber_fn(y_true,y_pred):
  error=y_true-y_pred
  is_small_error=tf.abs(error)<1
  squared_loss=tf.square(error)/2
  linear_loss=tf.abs(error)-0.5
  return tf.where(is_small_error,squared_loss,linear_loss)

In [35]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [45]:
input_shape=X_train.shape[1:]
model=keras.models.Sequential([
                               keras.layers.Dense(30,activation="selu",kernel_initializer="lecun_normal",input_shape=input_shape),
                               keras.layers.Dense(1)
])

In [46]:
model.compile(loss=huber_fn,optimizer="nadam",metrics=["mae"])

In [47]:
model.fit(X_train_scaled,y_train,epochs=2,batch_size=3,validation_data=(X_valid_scaled,y_valid))

Epoch 1/2
3870/3870 [==============================] - 6s 1ms/step - loss: 0.2577 - mae: 0.5586 - val_loss: 0.2976 - val_mae: 0.5801
Epoch 2/2
3870/3870 [==============================] - 5s 1ms/step - loss: 0.2013 - mae: 0.4900 - val_loss: 0.1868 - val_mae: 0.4688


##### 사용자 정의 요소를 가진 모델을 저장하고 로드하기
* 모델을 저장할때는 문제 없으나 로드할때는 함수 이름과 실제 함수를 매핑한 딕셔너리를 전달해야한다.

In [48]:
model.save("my_model_with_a_custom_loss.h5")

In [49]:
model=keras.models.load_model("my_model_with_a_custom_loss.h5",custom_objects={"huber_fn":huber_fn})

* thresh_hold 설정

In [52]:
def create_huber(threshold=1.0):
  def huber_fn(y_true,y_pred):
    error=y_true-y_pred
    is_small_error=tf.abs(error)<threshold
    square_loss=tf.square(error)/2
    linear_loss=threshold*tf.abs(error)-threshold**2/2
    return tf.where(is_small_error,square_loss,linear_loss)

In [53]:
model.compile(loss=create_huber(2.0),optimizer="nadam")

In [54]:
model.save("my_model_with_a_custom_loss_threshold.h5")

* 모델 저장할때 threshold값은 저장되지 않는다. 따라서 로드할때 threshold값을 지정해야한다.

In [56]:
model=keras.models.load_model("my_model_with_a_custom_loss_threshold.h5",custom_objects={"huber_fn":create_huber(2.0)})

* 위 문제는  keras.losses.Loss클래스를 상속하고 get_config()메서드를 구현하여 해결

In [61]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [62]:
input_shape=X_train.shape[1:]
model=keras.models.Sequential([
                               keras.layers.Dense(30,activation="selu",kernel_initializer="lecun_normal",input_shape=input_shape),
                               keras.layers.Dense(1)
])
model.compile(loss=HuberLoss(2.),optimizer="nadam",metrics=["mae"])
model.fit(X_train_scaled,y_train,epochs=2,validation_data=(X_valid_scaled,y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.9246 - mae: 0.9968 - val_loss: 0.3173 - val_mae: 0.5482
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2417 - mae: 0.5145 - val_loss: 0.2515 - val_mae: 0.5041


In [63]:
model.save("my_model_with_a_custom_loss_class.h5")

In [66]:
model=keras.models.load_model("my_model_with_a_custom_loss_class.h5",custom_objects={"HuberLoss":HuberLoss})

##### 활성화 함수,초기화 규제, 제한을 커스터마이징하기
* 손실,규제,제한,초기화,지표,활성화함수,층,모델과 같은 대부분의 케라스 기능은 유사하게 커스터마이징 가능

In [68]:
def my_softplus(z):
  return tf.math.log(tf.exp(z)+1.0)
def my_glorot_initializer(shape,dtype=tf.float32):
  stddev=tf.sqrt(2./(shape[0]+shape[1]))
  return tf.random.normal(shape,stddev=stddev,dtype=dtype)
def my_l1_regularizer(weights):
  return tf.reduce_sum(tf.abs(0.01*weights))
def my_positive_weights(weights):
  return tf.where(weights<0.,tf.zeros_lizer(weigths),weigths)

In [70]:
layer=keras.layers.Dense(30,activation=my_softplus,
                         kernel_initializer=my_glorot_initializer,
                         kernel_regularizer=my_l1_regularizer,
                         kernel_constraint=my_positive_weights)

* 함수가 모델과 함께 저장해야할 파라미터를 가지고 있다면 앞에와 같이 클래스를 상속해서 만든다.

In [77]:
class MyL1Regularizer(keras.regularizers.Regularizer):
  def __init__(self,factor):
    self.factor=factor
  def __call__(self,weights):
    return tf.reduce_sum(tf.abs(self.factor*weigths))
  def get_config(self):
    return {"factor":self.factor}

### 사용자 정의 지표
* 손실과 지표는 개념적으로 다른 것은 아니다.
* 손실은 모델 훈련을 위해 사용하므로 미분 가능하고 그래디언트가 코든 곳에서 0이 아니어야한다. 사람이 쉽게 이해 못해도 괜찮다.
* 지표는 이해하기 쉽고 미분 불가능에 그래디언트가 0이어도 괜찮다.
* 후버 손실함수는 지표로 사용해도 잘 동작한다.
* 훈련하는동안 각 배치에 대해 케라스는 지표를 계산하고 에포크가 시작할 때부터 평균을 기록한다.
* 정밀도는 양성과 거짓양성을 더한 갯수를 양성 개수로 나눈값.
* 배치마다 정밀도가 업데이트 되기에 이를 스티리밍 지표라고 한다.

In [83]:
precision=keras.metrics.Precision()
precision([0,1,0,1,1,0,1,1],[1,0,1,1,0,1,0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.25>

In [84]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.25>

In [85]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([3.], dtype=float32)>]

* 이런 스트리밍 지표를 만들고 싶다면 keras.metrics.Metric 클래스를 상속한다.

In [92]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [ ]:
m = HuberMetric(2.)

# total = 2 * |10 - 2| - 2²/2 = 14
# count = 1
# result = 14 / 1 = 14
m(tf.constant([[2.]]), tf.constant([[10.]]))

##### 사용자 정의 층
* 특이한 층을 가진 네트워크 만들기
* 가중치가 없는 층을 만들기 ex) Flatten,ReLu

In [96]:
exponential_layer=keras.layers.Lambda(lambda x: tf.exp(x))

In [97]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [98]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=input_shape),
    keras.layers.Dense(1),
    exponential_layer
])
model.compile(loss="mse", optimizer="sgd")
model.fit(X_train_scaled, y_train, epochs=5,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/5
363/363 [==============================] - 1s 2ms/step - loss: 1.0631 - val_loss: 0.4457
Epoch 2/5
363/363 [==============================] - 0s 1ms/step - loss: 0.4562 - val_loss: 0.3798
Epoch 3/5
363/363 [==============================] - 0s 1ms/step - loss: 0.4029 - val_loss: 0.3548
Epoch 4/5
363/363 [==============================] - 1s 1ms/step - loss: 0.3851 - val_loss: 0.3464
Epoch 5/5
162/162 [==============================] - 0s 826us/step - loss: 0.3586


0.3586340546607971

* 간단한 Dense층

In [119]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs): # 모든 하이퍼 파라미터를 매개변수로 받는다. **kwargs매개변수 중요.
        super().__init__(**kwargs)  # 부모 생성자를 호출하면서 kwargs를 전달해서 input_shape,trainable,name과 같은 기본 매개변수 처리
        self.units = units 
        self.activation = keras.activations.get(activation)
        # 위 함수는 객체나 "relu","selu"등을 받을수 있다/
    def build(self, batch_input_shape):# 가중치마다 add_weight메서드를 호출하여 층의 변수 만듬/
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # 반드시 끝에 존재해야한다.

    def call(self, X): # 이 층에 필요한 연산 수행한다/
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape): # 이층의 출력 크기를 반환한다.
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self): 
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [112]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [113]:
model = keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=input_shape),
    MyDense(1)
])

In [115]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.4904 - val_loss: 0.5329
Epoch 2/2
162/162 [==============================] - 0s 908us/step - loss: 0.3990


0.39900389313697815

In [116]:
model.save("my_model_with_a_custom_layer.h5")

In [117]:
model = keras.models.load_model("my_model_with_a_custom_layer.h5",
                                custom_objects={"MyDense": MyDense})

In [120]:
# 두개의 입력과 세개의 출력을 만드는 층.
# 함수형 API와 서브클래싱 API에만 사용가능
# 하나의 입력 출력을 쓰는 시퀀셜 API에서 불가능.
class MyMultiLayer(keras.layers.Layer):
  def call(self,x):
    x1,x2=x
    return [x1+x2,x1*x2,x1/x2]
  def compute_output_shape(self,batch_input_shape):
    b1,b2=batch_input_shape
    return [b1,b1,b1]

In [128]:
class MyGaussianNoise(keras.layers.Layer):
  def __init__(self,stddev,**kwargs):
    super().__init__(**kwargs)
    self.stddev=stddev
  def call(self,x,training=None):
    if training:
      noise=tf.random.normal(tf.shape(x),stddev=self.stddev)
      return x+noise
    else:
      return x
  def compute_output_shape(self,batch_input_shape):
    return batch_input_shape

In [129]:
model=keras.models.Sequential([
                               MyGaussianNoise(stddev=1.0),
                               keras.layers.Dense(30,activation="selu"),
                               keras.layers.Dense(1)
])

In [130]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 2.3285 - val_loss: 0.8946
Epoch 2/2
162/162 [==============================] - 0s 866us/step - loss: 0.7635


0.7635166645050049

### 사용자 정의 모델
* keras.Model클래스 상속

In [133]:
class ResidualBlock(keras.layers.Layer):
  def __init__(self,n_layers,n_neurons,**kwargs):
    super().__init__(**kwargs)
    self.hidden=[keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="he_normal") for _ in range(n_layers)]
  def call(self,inputs):
    Z=inputs
    for layer in self.hidden:
      Z=layer(Z)
    return inputs+Z

In [134]:
class ResidualRegressor(keras.Model):
  def __init__(self,output_dim,**kwargs):
    super().__init__(**kwargs)
    self.hidden1 = keras.layers.Dense(30,activation="elu",kernel_initializer="he_normal")
    self.block1=ResidualBlock(2,30)
    self.block2=ResidualBlock(2,30)
    self.out=keras.layers.Dense(output_dim)
  def call(self,inputs):
    Z=self.hidden1(inputs)
    for _ in range(1+3):
      Z=self.block1(Z)
    Z=self.block2(Z)
    return self.out(Z)

In [142]:
X_new_scaled = X_test_scaled
keras.backend.clear_session()
model=ResidualRegressor(1)
model.compile(loss="mse",optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5,batch_size=6)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Epoch 1/5
1935/1935 [==============================] - 4s 2ms/step - loss: 2.1859
Epoch 2/5
1935/1935 [==============================] - 3s 2ms/step - loss: 1.1448
Epoch 3/5
1935/1935 [==============================] - 3s 2ms/step - loss: 0.5320
Epoch 4/5
1935/1935 [==============================] - 3s 2ms/step - loss: 0.4874
Epoch 5/5
162/162 [==============================] - 0s 1ms/step - loss: 0.9722


#### 모델 구성요소에 기반한 손실과 지표
* 은닉층 제일 위에 보조출력을 놓고 재구성 손실.

In [154]:
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruct = keras.layers.Dense(8) # workaround for TF issue #46858
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")

    #Commented out due to TF issue #46858, see the note above
    #def build(self, batch_input_shape):
    #    n_inputs = batch_input_shape[-1]
    #    self.reconstruct = keras.layers.Dense(n_inputs)
    #    super().build(batch_input_shape)

    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)
        return self.out(Z)
model = ReconstructingRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2)
y_pred = model.predict(X_test_scaled)

Epoch 1/2
363/363 [==============================] - 8s 2ms/step - loss: 0.6849 - reconstruction_error: 0.6746
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.4070 - reconstruction_error: 0.3646


#### 자동 미분을 사용하여 그레디언트 계산하기

In [155]:
def f(w1,w2):
  return 3*w1**2+2*w1*w2

In [156]:
w1,w2=5,3
eps=1e-6
(f(w1+eps,w2)-f(w1,w2))//eps

36.0

In [158]:
w1,w2=tf.Variable(5.),tf.Variable(3.)
with tf.GradientTape() as tape:
  z=f(w1,w2)
gradients=tape.gradient(z,[w1,w2])

* GradientTape사용으로 자동미분을 진행한다.
* 이는 Variable형태만 사용 가능하다.

In [159]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

* gradient() 메서드가 호출 된 이후에는 테이프가 자동으로 삭제된다.
* gradient()메서드를 두번 호출하면 에러 발생한다.
* 어떤 텐서라도 감시하여 관련된 모든 연산을 기록하도록 강제가 가능하다.

In [161]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])

In [162]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

* stop_gradient를 사용하면 역전파가 안되게 막을 수 있다.


In [163]:
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
    z = f(w1, w2)

tape.gradient(z, [w1, w2])

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]